In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [3]:
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY= os.getenv("LANGSMITH_API_KEY")

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

d:\learning\learn_langgraph_ss\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [14]:
# from langchain_huggingface import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Gemma2-9b-It")
llm.invoke("Hello, world!")

AIMessage(content='Hello to you too! 👋  \n\nWhat can I do for you today? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_time': 0.036363636, 'prompt_time': 0.001989256, 'queue_time': 0.249912007, 'total_time': 0.038352892}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--de95034a-dd66-4840-8b2c-d120aebf6a86-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33})

In [7]:
while True:
    question = input("Type your question. If you want to quit the chat write quit")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("Goodbye take care of yourself")
        break

Goodbye take care of yourself


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [15]:
store = {}

In [16]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [17]:
config = {"configurable": {"session_id": "firstchat"}}

In [18]:
model_with_memory = RunnableWithMessageHistory(llm, get_session_history=get_session_history)

In [19]:
model_with_memory.invoke(("Hi! I am Vikas"), config=config).content

"Hello Vikas! 👋 \n\nIt's nice to meet you. What can I do for you today? 😊  \n\n"

In [20]:
model_with_memory.invoke(("Tell me what is my name?"), config=config).content

'Your name is Vikas.  \n\nI remember that from our first interaction! 😊  \n\n\n\nHow can I help you further?\n'

In [27]:
print(store['firstchat'])

Human: Hi! I am Vikas
AI: Hello Vikas! 👋 

It's nice to meet you. What can I do for you today? 😊  


Human: Tell me what is my name?
AI: Your name is Vikas.  

I remember that from our first interaction! 😊  



How can I help you further?



In [30]:
store['firstchat'].messages

[HumanMessage(content='Hi! I am Vikas', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Vikas! 👋 \n\nIt's nice to meet you. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 15, 'total_tokens': 44, 'completion_time': 0.052727273, 'prompt_time': 0.001929016, 'queue_time': 0.246577213, 'total_time': 0.054656289}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--498ae442-f522-421e-885b-3bfac20ebdf8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 29, 'total_tokens': 44}),
 HumanMessage(content='Tell me what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Vikas.  \n\nI remember that from our first interaction! 😊  \n\n\n\nHow can I help you further?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 58, 'tot

## RAG with LCEL

In [33]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [34]:
## Reading the txt files from source directory
loader = DirectoryLoader('data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

In [35]:
docs

[Document(metadata={'source': 'data\\llama3.txt'}, page_content='Llama (Large Language Model Meta AI) is a family of autoregressive large language models released by Meta AI starting in February 2023.[2][3] The latest version is Llama 3 released in April 2024.[4]\n\nModel weights for the first version of Llama were released to the research community under a non-commercial license.[5][3] Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use.[6][7] Llama models are trained at different parameter sizes, typically ranging between 7B and 70B.[4] Originally, Llama was only available as a foundation model.[8] Starting with Llama 2, Meta AI started releasing instruction fine-tuned versions alongside foundation models.[7]\n\nLlama models have been compared favorably against other large language models. Meta AI reported the original 13B parameter model\'s performance on most NLP benchmarks exceeded that of the much large

In [37]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(docs)

In [39]:
len(new_docs)

39

In [40]:
doc_strings = [doc.page_content for doc in new_docs]

In [ ]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [ ]:
# db = Chroma.from_documents(new_docs, embeddings)

In [ ]:
# retriever = db.as_retriever(search_kwargs={"k":4})

In [ ]:
# retriever.invoke("LLama")

[Document(metadata={'source': 'data\\llama3.txt'}, page_content='that permitted some commercial use.[6][7] Llama'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='versions of Llama were made accessible outside'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='Llama (Large Language Model Meta AI) is a family'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='is Llama 3 released in April 2024.[4]')]

In [47]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

d:\learning\learn_langgraph_ss\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vikas Chauhan\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fall

In [48]:
db = Chroma.from_documents(new_docs, embeddings)

In [49]:
retriever = db.as_retriever(search_kwargs={"k":4})

In [50]:
retriever.invoke("LLama")

[Document(metadata={'source': 'data\\llama3.txt'}, page_content='Llama models have been compared favorably against'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='Model weights for the first version of Llama were'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='7B and 70B.[4] Originally, Llama was only'),
 Document(metadata={'source': 'data\\llama3.txt'}, page_content='is Llama 3 released in April 2024.[4]')]

In [51]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [53]:
retrieval_chain = (
    RunnableParallel({"context":retriever, "question": RunnablePassthrough()}) | prompt | llm | StrOutputParser()
)

In [54]:
question = "What is llama3? Can you highlight 3 important points?"

In [55]:
print(retrieval_chain.invoke(question))

While this text snippet doesn't give a direct definition of Llama 3, we can glean some important points about it:

1. **Release Timing:** Llama 3 was released **sometime before April 2024**.
2. **Performance:**  Llama models, including Llama 3, have been **favorably compared** to other models, suggesting strong performance.
3. **Parameter Size:**  The text specifically mentions the **8B parameter version** of Llama 3, indicating different model sizes are available. 


Let me know if you'd like me to try finding more information about Llama 3! 



## Agents and Tools

In [56]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [57]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [58]:
tool.invoke("What is llama")

'Page: Llama (language model)\nSummary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 4, released in April 2025.\nLlama models come in different sizes, ranging from 1 billion to 2 trillion parameters. Initially only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.\nModel weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted some commercial use.\nAlongside the release of Llama 3, Meta added virtual assistant features to Facebook and WhatsApp in select regions, and a standalone website. Both services use a Llama 3 model.\n\n\n\nPage

In [59]:
from langchain_community.tools import YouTubeSearchTool
tool2 = YouTubeSearchTool()

In [60]:
tool2.invoke("3Blue1Brown")

"['https://www.youtube.com/watch?v=RQWpF2Gb-gU&pp=ygULM0JsdWUxQnJvd24%3D', 'https://www.youtube.com/watch?v=wjZofJX0v4M&pp=ygULM0JsdWUxQnJvd24%3D']"

In [64]:
from langchain_community.tools import DuckDuckGoSearchResults

In [66]:
tool3 = DuckDuckGoSearchResults()

In [67]:
tool3.invoke("India Pakistan War")

"snippet: ISLAMABAD, Pakistan (AP) — A gun massacre of tourists on April 22 pushed India and Pakistan a step closer to war, marking the biggest breakdown in relations since 2019. Conflict between India and Pakistan is not rare, with the two countries having periodically engaged in wars, clashes and skirmishes since gaining independence from British India in 1947., title: Nuclear rivals India and Pakistan step back from brink of war. Here's a ..., link: https://apnews.com/article/pakistan-india-tensions-timeline-kashmir-d43f29a59c31e2cf5e56c119aa098cb9, snippet: A war between India and Pakistan erupted over the Himalayan region, and in 1949, the two countries agreed to establish a ceasefire line dividing Kashmir, which is highly militarized and monitored ..., title: What's behind India and Pakistan's conflict over Kashmir, and why it's ..., link: https://abcnews.go.com/International/india-pakistan-kashmir-conflict-threat/story?id=121656628, snippet: New Delhi — A deadly terrorist attack

In [68]:
from langchain.agents import initialize_agent, AgentType, load_tools

In [70]:
tool = load_tools(["wikipedia"],llm=llm)

In [71]:
agent = initialize_agent(tool, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

C:\Users\Vikas Chauhan\AppData\Local\Temp\ipykernel_18816\612643009.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tool, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)


In [72]:
agent.run("What is the current GDP of India?")

C:\Users\Vikas Chauhan\AppData\Local\Temp\ipykernel_18816\4101983600.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is the current GDP of India?")




> Entering new AgentExecutor chain...
Thought: I need to look up the current GDP of India.
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. India

KeyboardInterrupt: 